In [1]:
# Boiler plate stuff to start the module
import jpype
import jpype.imports
from jpype.types import *
import sys 
import numpy as np
import traceback
import random
import Client
import copy
import operator

C:\Users\AStA\anaconda3\envs\gym\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\AStA\anaconda3\envs\gym\lib\site-packages\numpy\.libs\libopenblas.EL2C6PLE4ZYW3ECEVIV3OXXGRN2NRFM2.gfortran-win_amd64.dll
C:\Users\AStA\anaconda3\envs\gym\lib\site-packages\numpy\.libs\libopenblas.FB5AE2TYXYH2IJRDKGDGQ3XBKLKTF43H.gfortran-win_amd64.dll
C:\Users\AStA\anaconda3\envs\gym\lib\site-packages\numpy\.libs\libopenblas.WCDJNK7YVMPZQ2ME2ZZHJJRJ3JIKNDB7.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


In [2]:
# import the Java modules
from java.io import IOException
from java.net import Socket
from java.net import UnknownHostException

from javax.net.ssl import SSLSocket
from javax.net.ssl import SSLSocketFactory
from javax.xml.bind import JAXBException
from javax.xml.bind import UnmarshalException

from de.fhac.mazenet.server.generated import AwaitMoveMessageData
from de.fhac.mazenet.server.generated import Errortype
from de.fhac.mazenet.server.generated import MazeCom
from de.fhac.mazenet.server.generated import MazeComMessagetype
from de.fhac.mazenet.server.generated import MoveMessageData
from de.fhac.mazenet.server.generated import BoardData

from de.fhac.mazenet.server.networking import MazeComMessageFactory
from de.fhac.mazenet.server.networking import XmlInputStream
from de.fhac.mazenet.server.networking import XmlOutputStream

from de.fhac.mazenet.server.game import Board
from de.fhac.mazenet.server.game import Position
from de.fhac.mazenet.server.game import Card

In [3]:
HOST = "127.0.0.1"
PORT = 9888

In [4]:
import gym 
from gym import Env
from gym.spaces import Discrete, Box, Dict, Tuple, MultiBinary, MultiDiscrete
import numpy as np
import random
import os
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import VecFrameStack
from stable_baselines3.common.evaluation import evaluate_policy

In [5]:
from mss import mss
import pydirectinput
import cv2
import pytesseract
import matplotlib.pyplot as plt
import time

In [6]:
import functools

from pettingzoo import AECEnv, ParallelEnv
from pettingzoo.utils import agent_selector, parallel_to_aec, wrappers

In [7]:
ls_range = np.arange(1,6,2)
potentialShiftMoves =  []
potentialShiftMovesPos =  []
for i in ls_range:
    potentialShiftMoves.append((0, i))
    potentialShiftMoves.append((6, i))
    potentialShiftMoves.append((i, 0))
    potentialShiftMoves.append((i, 6))
    potentialShiftMovesPos.append(Position(0, i))
    potentialShiftMovesPos.append(Position(6, i))
    potentialShiftMovesPos.append(Position(i, 0))
    potentialShiftMovesPos.append(Position(i, 6))

In [8]:
class Agent_Client:
    def __init__(self, name, client=None):
        self.name = name
        self.client = client

In [9]:
class MARL_Env(AECEnv):
    
    metadata = {"render_modes": ["human"], "name": "rps_v2"}

    def __init__(self, render_mode=None, num_players=4):
        #self.manager = Client.Client("manager", HOST, PORT, True)
        #self.manager.login_manager()
        
        self.num_players = num_players
        self.possible_agents = [Agent_Client(name="player_" + str(r)) for r in range(self.num_players)]
        self.agent_name_mapping = dict(
            zip(self.possible_agents, list(range(len(self.possible_agents))))
        )

        # Gym spaces are defined and documented here: https://gym.openai.com/docs/#spaces
        self._action_spaces = {agent: MultiDiscrete([12, 4, 7, 7], dtype=np.int8) for agent in self.possible_agents}
        self._observation_spaces = {
            agent: Tuple((Box(low=0, high=6, shape=(1,2), dtype=np.uint8),
                           Box(low=0, high=6, shape=(1,2), dtype=np.uint8)
                           ))  for agent in self.possible_agents
        }
        self.render_mode = render_mode
        self.player_pos = {agent: Box(low=0, high=6, shape=(1,2)) for agent in self.possible_agents}
        self.next_treasure_pos = {agent: Box(low=0, high=6, shape=(1,2)) for agent in self.possible_agents}
        self.is_done = False
        

    # this cache ensures that same space object is returned for the same agent
    # allows action space seeding to work as expected
    @functools.lru_cache(maxsize=None)
    def observation_space(self, agent):
        # Gym spaces are defined and documented here: https://gym.openai.com/docs/#spaces
        return Tuple((Box(low=0, high=6, shape=(1,2), dtype=np.uint8),
                           Box(low=0, high=6, shape=(1,2), dtype=np.uint8)
                           ))

    @functools.lru_cache(maxsize=None)
    def action_space(self, agent):
        return MultiDiscrete([12, 4, 7, 7], dtype=np.int8)

    def render(self):
        pass

    def observe(self, agent):
        """
        Observe should return the observation of the specified agent. This function
        should return a sane observation (though not necessarily the most up to date possible)
        at any time after reset() is called.
        """
        pass

    def close(self):
        """
        Close should release any graphical displays, subprocesses, network connections
        or any other environment data which should not be kept around after the
        user is no longer using the environment.
        """
        pass

    def reset(self, seed=None, return_info=False, options=None):
        """
        Reset needs to initialize the following attributes
        - agents
        - rewards
        - _cumulative_rewards
        - terminations
        - truncations
        - infos
        - agent_selection
        And must set up the environment so that render(), step(), and observe()
        can be called without issues.
        Here it sets up the state dictionary which is used by step() and the observations dictionary which is used by step() and observe()
        """
        #self.manager.stop_game(self.num_players)
        #self.manager.start_game(self.num_players)
        
        pydirectinput.click(x=1050, y=150)
        # start
        pydirectinput.click(x=950, y=150)
        time.sleep(1)
        for agent in self.possible_agents:
            while True:
                agent.client = Client.Client(agent.name, HOST, PORT, True)
                try:
                    login = MazeComMessageFactory.createLoginMessage(agent.name)
                    agent.client.out.write(login)
                    receivedMazeCom = agent.client.in_.readMazeCom()
                    if(receivedMazeCom.getMessagetype() == MazeComMessagetype.LOGINREPLY):
                        newid = receivedMazeCom.getLoginReplyMessage().getNewID()
                        print("newid:", newid)
                        agent.client.setId(newid)
                        break
                    else:
                        print(receivedMazeCom.getAcceptMessage().getErrortypeCode())
                except:
                    print("failed login :)")
        #self.possible_agents = [Client.Client("player_" + str(r), HOST, PORT, True) for r in range(self.num_players+1)]
        #self.possible_agents = self.possible_agents[1:]
        
        self.is_done = False
        for i in self.possible_agents:
            print("id:", i.client.id_)
        #print("agents:", self.agents)
        self.possible_agents = sorted(self.possible_agents, key=operator.attrgetter('client.id_'))
        for i in self.possible_agents:
            print("id:", i.client.id_)
            
        self.agents = self.possible_agents[:]
        self.rewards = {agent: 0 for agent in self.agents}
        self._cumulative_rewards = {agent: 0 for agent in self.agents}
        self.terminations = {agent: False for agent in self.agents}
        self.truncations = {agent: False for agent in self.agents}
        self.infos = {agent: {} for agent in self.agents}
        self.state = {agent: None for agent in self.agents}
        self.observations = {agent: None for agent in self.agents}
        self.num_moves = 0
        
        """
        Our agent_selector utility allows easy cyclic stepping through the agents list.
        """
        self._agent_selector = agent_selector(self.agents)
        self.agent_selection = self._agent_selector.next()
        print("end reset")
            

                
    def moeglicheOrientierungen(self, rot, card):
        card_ = Card(card)
        openings = card_.getOpenings()
        if(card.getShape() == Card.CardShape.I):
            openings.setBottom(not openings.isBottom())
            openings.setLeft(not openings.isLeft())
            openings.setRight(not openings.isRight())
            openings.setTop(not openings.isTop())
            return  Card(card_)
        else:
            if(rot == 0):
                return Card(card.getShape(), Card.Orientation.D0, card.getTreasure())
            if(rot == 1):
                return Card(card.getShape(), Card.Orientation.D90, card.getTreasure())
            if(rot == 2):
                return Card(card.getShape(), Card.Orientation.D180, card.getTreasure())
            if(rot == 3):
                return Card(card.getShape(), Card.Orientation.D270, card.getTreasure())
        return Card(card.getShape(), Card.Orientation.D0, card.getTreasure())
    
    def Move(self, awaitMove, action):
        reward = 0
        boardData = awaitMove.getBoard()
        treasure = awaitMove.getTreasureToFindNext()
        board = Board(boardData)
        playerPosition = board.findPlayer(self.agent_selection.client.getId())
        psm = potentialShiftMoves[action[0]]
        position = Position(psm[0], psm[1])
        
        potentialMove = MoveMessageData()
        psmp = copy.copy(potentialShiftMovesPos)
        if(position == board.getForbidden()):
            #print("forbidden shift position, random shift position used, reward decreased")
            reward -= 10
            try:
                psmp.remove(position)
            except Exception:
                print("element not in list, skipping")
            potentialMove.setShiftPosition(psmp[0])
        else:
            potentialMove.setShiftPosition(position)
        
        treasurePositionData = board.findTreasure(treasure)
        
        orientedShiftCard = self.moeglicheOrientierungen(action[1], board.getShiftCard())
        
        potentialMove.setShiftCard(orientedShiftCard)
        potentialMove.setNewPinPos(playerPosition)

        boardNext = board.fakeShift(potentialMove)
        treasurePositionData = boardNext.findTreasure(treasure)
        
        new_player_pos = boardNext.findPlayer(self.agent_selection.client.getId())
        reachablePositions = boardNext.getAllReachablePositions(new_player_pos)
        rp = np.array(reachablePositions)
        chosen_pos_obj = Position(action[2], action[3])
        chosen_pos_arr = np.array(action[2], action[3])
        if(treasurePositionData == None):
            #print("no treasure? skipping")
            potentialMove.setNewPinPos(new_player_pos)
            self.next_treasure_pos[self.agent_selection] = treasurePositionData
            self.player_pos[self.agent_selection] = new_player_pos
            return potentialMove, reward

        if(chosen_pos_obj not in rp):
            #print("cannot reach position, reward decreased")
            potentialMove.setNewPinPos(new_player_pos)
            self.next_treasure_pos[self.agent_selection] = treasurePositionData
            self.player_pos[self.agent_selection] = new_player_pos
            return potentialMove, reward - 10
        else:
            #print("treasure position data:", treasurePositionData)
            treasurePosition = Position(treasurePositionData)
            #print("treasure pos:", treasurePosition)

            if(chosen_pos_obj == treasurePosition):
                #print("straight to the treasure, reward increased!")
                potentialMove.setNewPinPos(treasurePosition)
                self.next_treasure_pos[self.agent_selection] = treasurePositionData
                self.player_pos[self.agent_selection] = treasurePosition
                return potentialMove, reward + 20
            else:
                #print("reachable position, but no treasure")
                potentialMove.setNewPinPos(chosen_pos_obj)
                self.next_treasure_pos[self.agent_selection] = treasurePositionData
                self.player_pos[self.agent_selection] = chosen_pos_obj
                return potentialMove, reward - 5

        #print("Object not reachable: random move :/")
        potentialMove.setNewPinPos(reachablePositions[0])
        self.next_treasure_pos[self.agent_selection] = treasurePositionData
        self.player_pos[self.agent_selection] = reachablePositions[0]
        return potentialMove, reward - 10
    
    def awaitMove(self, receivedMazeCom, action):
        awaitMove = receivedMazeCom.getAwaitMoveMessage()
        move, reward = self.Move(awaitMove, action)
        mazeComToSend = MazeCom()
        mazeComToSend.setId(self.agent_selection.client.getId())
        mazeComToSend.setMessagetype(MazeComMessagetype.MOVE)
        mazeComToSend.setMoveMessage(move)
        self.agent_selection.client.out.write(mazeComToSend)
        return reward                
                
    def step(self, action):
        """
        step(action) takes in an action for the current agent (specified by
        agent_selection) and needs to update
        - rewards
        - _cumulative_rewards (accumulating the rewards)
        - terminations
        - truncations
        - infos
        - agent_selection (to the next agent)
        And any internal state used by observe() or render()
        """
        print("begin step")
        if self.is_done:
            self.terminations[self.agent_selection] = True
        else:
            reward = 0
            try:
                receivedMazeCom = self.agent_selection.client.in_.readMazeCom()
                if(receivedMazeCom.getMessagetype() == MazeComMessagetype.LOGINREPLY):
                    newid = receivedMazeCom.getLoginReplyMessage().getNewID()
                    self.agent_selection.client.setId(newid)
                elif(receivedMazeCom.getMessagetype() == MazeComMessagetype.ACCEPT):
                    print(receivedMazeCom.getAcceptMessage().getErrortypeCode())
                    reward += 2
                elif (receivedMazeCom.getMessagetype() == MazeComMessagetype.DISCONNECT):
                    print(receivedMazeCom.getDisconnectMessage().getErrortypeCode())
                    reward -= 10
                elif(receivedMazeCom.getMessagetype() == MazeComMessagetype.AWAITMOVE):
                    print(action)
                    reward += self.awaitMove(receivedMazeCom, action)
                elif(receivedMazeCom.getMessagetype() == MazeComMessagetype.MOVEINFO):
                    #print("in MoveInfo")
                    x = 0
                elif(receivedMazeCom.getMessagetype() == MazeComMessagetype.WIN):
                    print("You have won")
                    self.is_done = True
                    reward += 100
                else:
                    print("Unknown message type: " + receivedMazeCom.getMessagetype())
            except Exception as e:
                print(traceback.format_exc())
                if str(type(e)) == "<java class 'java.net.SocketException'>":
                    print("socket error, disconnected")
                    sys.exit(1)
                elif str(type(e)) == "<java class 'java.io.EOFException'>":
                    print("end reached, disconnecting")
                    sys.exit(1)
            self.rewards[self.agent_selection] = reward
            self._cumulative_rewards[self.agent_selection] += reward
            self.observations[self.agent_selection] = (self.player_pos[self.agent_selection], self.next_treasure_pos[self.agent_selection])
            if self.is_done:
                self.terminations[self.agent_selection] = True
            else:
                self.terminations[self.agent_selection] = False
            self.agent_selection = self._agent_selector.next()


In [10]:
import AgentNN

import torch
import torch.nn as nn
import torch.optim as optim
from torch.distributions.categorical import Categorical

In [11]:
path = 'Training/Logs'
#model = PPO('MlpPolicy', env, verbose=1, tensorboard_log=path)

In [12]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
ent_coef = 0.1
vf_coef = 0.1
clip_coef = 0.1
gamma = 0.99
batch_size = 32
stack_size = 4
frame_size = (64, 64)
max_cycles = 125
total_episodes = 2

""" ENV SETUP """
env = MARL_Env(num_players=4)
num_agents = len(env.possible_agents)
num_actions = np.prod(env.action_space(env.possible_agents[0]).nvec) # env.action_space(env.possible_agents[0]).n
observation_size = env.observation_space(env.possible_agents[0]).shape

""" LEARNER SETUP """
agent = AgentNN.Agent(num_actions=num_actions).to(device)
optimizer = optim.Adam(agent.parameters(), lr=0.001, eps=1e-5)

""" ALGO LOGIC: EPISODE STORAGE"""
end_step = 0
total_episodic_return = 0
rb_obs = torch.zeros((max_cycles, num_agents, stack_size, *frame_size)).to(device)
rb_actions = torch.zeros((max_cycles, num_agents)).to(device)
rb_logprobs = torch.zeros((max_cycles, num_agents)).to(device)
rb_rewards = torch.zeros((max_cycles, num_agents)).to(device)
rb_terms = torch.zeros((max_cycles, num_agents)).to(device)
rb_values = torch.zeros((max_cycles, num_agents)).to(device)

In [13]:
""" TRAINING LOGIC """
# train for n number of episodes
for episode in range(total_episodes):

    # collect an episode
    with torch.no_grad():

        # collect observations and convert to batch of torch tensors
        next_obs = env.reset()
        # reset the episodic return
        total_episodic_return = 0

        # each episode has num_steps
        for step in range(0, max_cycles):

            # rollover the observation
            obs = AgentNN.batchify_obs(next_obs, device)

            # get action from the agent
            actions, logprobs, _, values = agent.get_action_and_value(obs)

            # execute the environment and log data
            next_obs, rewards, terms, truncs, infos = env.step(
                AgentNN.unbatchify(actions, env)
            )

            # add to episode storage
            rb_obs[step] = obs
            rb_rewards[step] = AgentNN.batchify(rewards, device)
            rb_terms[step] = AgentNN.batchify(terms, device)
            rb_actions[step] = actions
            rb_logprobs[step] = logprobs
            rb_values[step] = values.flatten()

            # compute episodic return
            total_episodic_return += rb_rewards[step].cpu().numpy()

            # if we reach termination or truncation, end
            if any([terms[a] for a in terms]) or any([truncs[a] for a in truncs]):
                end_step = step
                break

    # bootstrap value if not done
    with torch.no_grad():
        rb_advantages = torch.zeros_like(rb_rewards).to(device)
        for t in reversed(range(end_step)):
            delta = (
                rb_rewards[t]
                + gamma * rb_values[t + 1] * rb_terms[t + 1]
                - rb_values[t]
            )
            rb_advantages[t] = delta + gamma * gamma * rb_advantages[t + 1]
        rb_returns = rb_advantages + rb_values

    # convert our episodes to batch of individual transitions
    b_obs = torch.flatten(rb_obs[:end_step], start_dim=0, end_dim=1)
    b_logprobs = torch.flatten(rb_logprobs[:end_step], start_dim=0, end_dim=1)
    b_actions = torch.flatten(rb_actions[:end_step], start_dim=0, end_dim=1)
    b_returns = torch.flatten(rb_returns[:end_step], start_dim=0, end_dim=1)
    b_values = torch.flatten(rb_values[:end_step], start_dim=0, end_dim=1)
    b_advantages = torch.flatten(rb_advantages[:end_step], start_dim=0, end_dim=1)

    # Optimizing the policy and value network
    b_index = np.arange(len(b_obs))
    clip_fracs = []
    for repeat in range(3):
        # shuffle the indices we use to access the data
        np.random.shuffle(b_index)
        for start in range(0, len(b_obs), batch_size):
            # select the indices we want to train on
            end = start + batch_size
            batch_index = b_index[start:end]

            _, newlogprob, entropy, value = agent.get_action_and_value(
                b_obs[batch_index], b_actions.long()[batch_index]
            )
            logratio = newlogprob - b_logprobs[batch_index]
            ratio = logratio.exp()

            with torch.no_grad():
                # calculate approx_kl http://joschu.net/blog/kl-approx.html
                old_approx_kl = (-logratio).mean()
                approx_kl = ((ratio - 1) - logratio).mean()
                clip_fracs += [
                    ((ratio - 1.0).abs() > clip_coef).float().mean().item()
                ]

            # normalize advantaegs
            advantages = b_advantages[batch_index]
            advantages = (advantages - advantages.mean()) / (
                advantages.std() + 1e-8
            )

            # Policy loss
            pg_loss1 = -b_advantages[batch_index] * ratio
            pg_loss2 = -b_advantages[batch_index] * torch.clamp(
                ratio, 1 - clip_coef, 1 + clip_coef
            )
            pg_loss = torch.max(pg_loss1, pg_loss2).mean()

            # Value loss
            value = value.flatten()
            v_loss_unclipped = (value - b_returns[batch_index]) ** 2
            v_clipped = b_values[batch_index] + torch.clamp(
                value - b_values[batch_index],
                -clip_coef,
                clip_coef,
            )
            v_loss_clipped = (v_clipped - b_returns[batch_index]) ** 2
            v_loss_max = torch.max(v_loss_unclipped, v_loss_clipped)
            v_loss = 0.5 * v_loss_max.mean()

            entropy_loss = entropy.mean()
            loss = pg_loss - ent_coef * entropy_loss + v_loss * vf_coef

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

    y_pred, y_true = b_values.cpu().numpy(), b_returns.cpu().numpy()
    var_y = np.var(y_true)
    explained_var = np.nan if var_y == 0 else 1 - np.var(y_true - y_pred) / var_y

    print(f"Training episode {episode}")
    print(f"Episodic Return: {np.mean(total_episodic_return)}")
    print(f"Episode Length: {end_step}")
    print("")
    print(f"Value Loss: {v_loss.item()}")
    print(f"Policy Loss: {pg_loss.item()}")
    print(f"Old Approx KL: {old_approx_kl.item()}")
    print(f"Approx KL: {approx_kl.item()}")
    print(f"Clip Fraction: {np.mean(clip_fracs)}")
    print(f"Explained Variance: {explained_var.item()}")
    print("\n-------------------------------------------\n")

newid: 3
newid: 1
newid: 4
newid: 2
id: 3
id: 1
id: 4
id: 2
id: 1
id: 2
id: 3
id: 4
end reset


TypeError: 'NoneType' object is not iterable

In [ ]:
""" RENDER THE POLICY """
env = MARL_Env(num_players=4)

agent.eval()

with torch.no_grad():
    # render 5 episodes out
    for episode in range(5):
        obs = env.reset()
        terms = [False]
        truncs = [False]
        while not any(terms) and not any(truncs):
            actions, logprobs, _, values = agent.get_action_and_value(obs)
            obs, rewards, terms, truncs, infos = env.step(AgentNN.unbatchify(actions, env))
            obs = AgentNN.batchify_obs(obs, device)
            terms = [terms[a] for a in terms]
            truncs = [truncs[a] for a in truncs]

In [ ]:
ppo_path = "Training/Saved Models/PPO_100_Mazenet_Model"
model.save(ppo_path)

In [ ]:
from pettingzoo.test import api_test
from pettingzoo.test import parallel_api_test

In [ ]:
env = MARL_Env(num_players=4)
#api_test(env, num_cycles=100, verbose_progress=True)
env.reset()
#print(type(sorted(env.agent_iter(), key=operator.attrgetter('id_'))))
#print(env.agent_iter())
for agent in env.agent_iter():
    #print("agent:", agent.name)
    observation, reward, termination, truncation, info = env.last()
    action = env.action_space(agent).sample()
    # time.sleep(3)
    env.step(action)